In [1]:
from snappy import ProductIO
from snappy import jpy
from snappy import GPF

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline

import numpy as np

from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from datetime import timedelta
from datetime import datetime

import zipfile
import os 
import time
import shutil
import pickle

In [2]:
# télecharge les images correspndantes aux critères de recherche

def download_olci_data(user,password,url,footprint,datestart,dateend,zip_file_path,image_number):
    
    #user = 'jonathanc'
    #password = 'Tutorat2019'
    #api = SentinelAPI(user, password, 'https://scihub.copernicus.eu/dhus')
    #url = 'https://coda.eumetsat.int'
    api = SentinelAPI(user, password, url)

    #footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
    #date = ('20191001','NOW')
    products = api.query(footprint,
                         date = (datestart,dateend),
                         platformname = 'Sentinel-3',
                         area_relation='Intersects', 
                         producttype='OL_2_WFR___',
                         order_by='beginposition',
                         #timeliness='Non Time Critical',
                         instrumentshortname = 'OLCI',
                         onlinequalitycheck = 'PASSED',
                         productlevel = 'L2')
    # convert to Pandas DataFrame
    products_df = api.to_dataframe(products)
    # sort and limit to first 5 sorted products
    products_df_sorted = products_df.sort_values(['beginposition'], ascending=[True])
    products_df_sorted = products_df_sorted.head(image_number)
    # download sorted and reduced products
    api.download_all(products_df_sorted.index,directory_path = zip_file_path+"/")
    print zip_file_path+"/"
    #zip_file_path = "/home/courtois/jupyter-sentinel"
    file_list = os.listdir(zip_file_path+"/")
    # select the directory where the files are and where you want to unzip it
    abs_path = []
    # list for the .zip file to unzip
    c=0
    for a in file_list:
        if a.endswith(".zip"):  # select only the .zip files in the folder
            x = zip_file_path+'/'+a
            c += 1
            print x
            abs_path.append(x)  # put the .zip file in the list
    for f in abs_path:
        zip=zipfile.ZipFile(f)
        zip.extractall(zip_file_path)  # unzip the file in the zip_file_path
        os.remove(f)   # delet the .zip file
        
    print("File in :" + zip_file_path)
    print("DONE")

In [3]:
# donne une liste de fichier en affichant que les nouveaux fichiers si images_list n'es pas une liste vide.
def list_product(images_path, images_list):
    file_list = os.listdir(images_path +"/")
    # definition du chemin d'accès

    # list contenant tout les noms de fichier 
    
    add_name_list = []
    for a in file_list:
        x = images_path+'/'+a
        #print x
        if a.endswith(".SEN3") and not(x in images_list):
            add_name_list.append(x)  # Dézip
    return add_name_list

# exemple d'utilisation
#images_path = "/media/DATA-4To/Sentinel"
#images_list = []
#images_list += list_product(images_path, images_list)
#print len(images_list),"files"

In [4]:
# Créé un filtre (liste d'index) de taille déterminé en partant d'un point de coordonée
def cubixGeoCut(point_lon,point_lat,sub_product,band_product,width, height):
    Lon = sub_product.getBand('longitude');
    Lat = sub_product.getBand('latitude');
    # on obtient les matrices de longitude et latitude
    
    band_Width = band_product.getRasterWidth()
    band_Height = band_product.getRasterHeight()
    
    Lon_d = np.zeros(band_Width*band_Height, dtype = np.float32)
    Lon.readPixels(0,0,band_Width,band_Height,Lon_d)
    Lon_d.shape = (band_Height, band_Width) 

    Lat_d = np.zeros(band_Width*band_Height, dtype = np.float32)
    Lat.readPixels(0,0,band_Width,band_Height,Lat_d)
    Lat_d.shape = (band_Height, band_Width) 

    # on chercher le point le plus  proche de la coordonnées souhaitée
    min_lon_mat = abs(Lon_d - point_lon)
    min_lat_mat = abs(Lat_d - point_lat)

    #obtention de l'indice du minimum
    min_coord = np.dstack((min_lon_mat,min_lat_mat))
    min_coord = np.dstack((min_coord,min_coord[:,:,0]+min_coord[:,:,1]))
    minc = np.unravel_index(np.argmin(min_coord[:,:,2], axis=None), min_coord[:,:,0].shape)

    # création de la matrice d'index de taille 125*125 partant du point de coordonné haut gauche
    x=0
    submatidx = np.zeros((height*width,2),dtype=int)

    for i in range(minc[0],minc[0]+height):
        for j in range(minc[1],minc[1]+width):
            submatidx[x] = (i,j)
            x = x+1
    return submatidx

In [5]:
# créé une liste des noms de bandes
def reflectance_band_names():
    reflect_band_names = []
    #recupréation du nom des bandes interssantes
    for i in range(1,10):
        reflect_band_names.append(str('Oa0'+str(i)+'_reflectance'))
    reflect_band_names.append(str('Oa10_reflectance'))
    reflect_band_names.append(str('Oa11_reflectance'))
    reflect_band_names.append(str('Oa12_reflectance'))
    reflect_band_names.append(str('Oa16_reflectance'))
    reflect_band_names.append(str('Oa17_reflectance'))
    reflect_band_names.append(str('Oa18_reflectance'))
    reflect_band_names.append(str('Oa21_reflectance'))
    #print(reflect_band_names)
    return reflect_band_names

In [6]:
def date_finder(name,tag):
    # recupération de l'index du debut de la date dans le nom ex : ...WFR____20191026T100707...
    date_index = name.find(tag)+len(tag)
    # extraction de la date : 20191026
    date_str = name[date_index:date_index+8]
    # mise en forme jour, mois, années : ['26', '10', '2019']
    date = [date_str,date_str[6:8],date_str[4:6],date_str[0:4]]
    return date

In [22]:
def data_image_builder(images_list, footprint, point_coord, image_size,cloud_cover=5, print_match=False):
    # variables init
    imageData = []
    prog = 1
    point_lon = point_coord[0]
    point_lat = point_coord[1] 
    height_reduit = image_size[0]
    width_reduit = image_size[1]
    # réglage de la géométrie et des sous produit
    SubsetOp = jpy.get_type('org.esa.snap.core.gpf.common.SubsetOp')
    WKTReader =jpy.get_type('com.vividsolutions.jts.io.WKTReader')
    geometry = WKTReader().read(footprint) #préparation de la géométrie de la ZOI 
    
    # liste des noms de bande
    reflect_band_names = reflectance_band_names()
    print('refelct band name\n')
    print(reflect_band_names)
    for path_name in images_list:
        print(path_name)

        # chargement du product
        product = ProductIO.readProduct(path_name)

        HashMap = jpy.get_type('java.util.HashMap')  
        parameters = HashMap()

        # generation d'un sous produit autour de la zoi
        op = SubsetOp()
        op.setSourceProduct(product)
        op.setGeoRegion(geometry)
        sub_product = op.getTargetProduct()

        # Obtention d'une liste de bandes 
        reflect_band_data = []
        timesaver = True
        for name in reflect_band_names:

            start_time = time.time()
            reflect_band_data.append(sub_product.getBand(name)) 
            if (timesaver):
                # Récupération de la tailles des bandes
                Width = reflect_band_data[0].getRasterWidth()
                Height = reflect_band_data[0].getRasterHeight()
                print "Image full scale : Width:",Width," ; Height:",Height
                if not(Width>2*width_reduit and Height>2*height_reduit):

                    print(Width,width_reduit,Height,height_reduit)
                    print("--- %s seconds size break---" % (time.time() - start_time))
                    break
                else:
                    timesaver = False
        # outils de filtrage :
        # on filtre si : la taille ne correspond pas OU si la couerture T/C est > 10 %
        remove = True

        # test de taille minimum pour l'image
        if (Width>2*width_reduit and Height>2*height_reduit):
            # extraire les données de reflectances de toutes les bandes dans une matrice
            # definition des paramètres codant la découpe de l'image dans la ZOI
            # point_lon = 4.654875873875496 
            # point_lat = 43.41241369162426
            # height_reduit = 125
            # width_reduit = 125
            # matrice de donnée
            # IL FAUDRA surement verifier les index pour ne pas depasser les capacitée de la matrice ######################
            #print "len(reflect_band_data)",len(reflect_band_data)
            
            data_reflect = np.empty([len(reflect_band_data), width_reduit, height_reduit ])
            # etablissement de la list d'indice qui permetera de reduire la matrice d'image de la ZOI
            submatidx = cubixGeoCut(point_lon,point_lat,sub_product,reflect_band_data[0],width_reduit, height_reduit )
            #print("submatidx",submatidx)

            multiSpectralMatrixTmp = []
            for i in range(1,len(reflect_band_data)):

                # extraction des données 
                reflect_data = np.zeros(Width*Height, dtype = np.float32)
                reflect_band_data[i].readPixels(0,0,Height,Width,reflect_data)
                reflect_data.shape = Width, Height

                print('index')
                print(submatidx[:,0])
                print(reflect_data.shape)
                # reduction de la donnée dans la zone definit
                data_reflect[i] = reflect_data[submatidx[:,0],submatidx[:,1]].reshape(height_reduit,width_reduit)

                # afficher la coverture terrestre et nuageuse
                c=0
                for j in range(0,data_reflect[i].shape[0]):
                    for k in range(0,data_reflect[i].shape[1]):
                        if (data_reflect[i][j,k] > 1):
                            c = c+1
                c = (c*100)/(data_reflect[i].shape[0]*data_reflect[i].shape[1])
                if (i == 0) : print "cloud/land coverage:",c,"%"

                if (c<=cloud_cover):
                    # ici on est dans la condition couverture L/C <= 5% ET taille de l'image de base convenable
                    remove = False
                    multiSpectralMatrixTmp.append(data_reflect[i])
                else:
                    i = len(reflect_band_data)+1
                    print("--- %s seconds reading band break ---" % (time.time() - start_time))
                    break

                # plot et subplot etc ...
                if (i==5 and not(remove) and print_match):

                    # plot
                    plt.figure(figsize=(8, 8))                 # adjusting the figure window size
                    fig = plt.imshow(data_reflect[i], cmap = cm.gray)  #matplotlib settings for the current image
                    fig.axes.get_xaxis().set_visible(False)
                    fig.axes.get_yaxis().set_visible(False)
                    plt.show()
                    # plot
                    plt.figure(figsize=(8, 8))                 # adjusting the figure window size
                    fig = plt.imshow(reflect_data, cmap = cm.gray)  #matplotlib settings for the current image
                    fig.axes.get_xaxis().set_visible(False)
                    fig.axes.get_yaxis().set_visible(False)
                    plt.show()

        product.closeIO() 
        # si l'iùage n'as pas passée les conditions ou si elle ne possèdent pas ses 16 bandes exploitable on la supprime
        if remove or (len(multiSpectralMatrixTmp)!=16):
            print('removed')
            shutil.rmtree(path_name)
        else :
            # ajout de la date en fin de list
            date = date_finder(path_name,'WFR____')
            multiSpectralMatrixTmp.append(date)
            imageData.append(multiSpectralMatrixTmp)
            print("--- %s seconds Processing done ---" % (time.time() - start_time))
        print prog,"/",len(images_list)
        prog +=1
    return imageData

In [23]:
def add_one_day_imlistata(imageList):
    date_list = []
    for i in range(len(imageList)): 
        date_list.append(date_finder(imageList[i],'WFR____')[0])
    date_list.sort(reverse = True)
    
    addtime = (datetime.strptime(date_list[0], '%Y%m%d')+timedelta(hours=+1)).__str__()
    addtime = addtime[0:4]+addtime[5:7]+addtime[8:10]

    return addtime

In [24]:
def build_data_base(acount,url,footprint, point_coord, Dates,images_path, image_size, image_number, Download_step, Download = True):
    user = acount[0]
    password = acount[1]
    datestart = Dates[0]
    dateend = Dates[1]
    image_segmentation = 0
    images_list = []
    imageData = []
    stopDL = False

    while (image_number-image_segmentation > 0):
        # segmentation des données pour le téléchargement
        image_segmentation += Download_step

        if not(stopDL):
            # Téléchargement des données
            if (Download):
                download_olci_data(user,password,url,footprint,datestart,dateend,images_path,Download_step)

            # Récupération des noms de fichiers fraichement téléchargé dans un segment
            new_images_list = list_product(images_path , images_list)

        if new_images_list != [] :
            # Mise à jour de la liste total des fichiers téléchargés
            images_list += new_images_list;

            # Création d'une liste de matrice de dimension image_size_x,image_size_y,nombre_de_raie
            imageData += data_image_builder(new_images_list, footprint, point_coord, image_size)

            # Récupération de la date du dernier fichier téléchargé 
            #  pour definir la date des prochaines images à télécharger
            datestart = add_one_day_imlistata(new_images_list)

            # mise en format datetime des date de debut et fin pour pouvoir les comparer
            start_datetime = datetime.strptime(datestart, '%Y%m%d')

            if (dateend == ('NOW')):
                date_end = datetime.today()
            else:
                date_end = datetime.strptime(dateend, '%Y%m%d')

            print (start_datetime,date_end)
            if (start_datetime > date_end):
                image_segmentation = image_number+1
        else:
            # il n'y à plus de fichier à télécharger on stop le télechargement
            stopDL = True 
            image_segmentation = image_number

        print(" ------------ %d download / %d ------------" % (image_segmentation, image_number))

    print("TASK DONE\nCreating ImageDataMatrix file ...")  
    with open(images_path+"/ImageDataMatrix.txt", "wb") as fp:   #Pickling
        pickle.dump(imageData, fp)
    print"DONE: ImageDataMatrix.txt saved in directory:\n"+images_path+'\n'
    
    print'Unpickling with:\nwith open("ImageDataMatrix.txt", "rb") as fp:\n\tdata = pickle.load(fp)'

In [25]:
user = 'jonathanc'
password = 'Tutorat2019'
url = 'https://coda.eumetsat.int'
# delta du rhone
#footprint = 'POLYGON ((3.632678257283426 44.218520813901634, 5.593941712716586 43.91153565143112,5.122306465193042 42.40131912218008, 3.208397257788463 42.706154058381806,3.632678257283426 44.218520813901634))'
#australie mundo island
footprint = 'POLYGON ((138.3448183694628 -34.380583465883916, 142.6707903040077 -35.47472630122478,141.1587354250971 -39.6129906209986, 136.6174782211944 -38.46029724665446,138.3448183694628 -34.380583465883916))'
datestart = ('20190101')
dateend = ('NOW')
#images_path = "/media/DATA-4To/Sentinel/Delta_rhone"
images_path = "/media/DATA-4To/Sentinel/Australie"
image_number = 1000
Download_step = 10

#delta du rhone
#point_coord = (4.654875873875496,43.41241369162426)
#Australie
point_coord = (138.81860602586, -35.455073438277715)
image_size = (125,125)

acount = [user,password]
Dates = [datestart,dateend]

In [26]:
build_data_base(acount,url,footprint, point_coord, Dates,images_path, image_size, image_number, Download_step,Download=True)

Querying products: 100%|██████████| 258/258 [00:00<00:00, 420.02 products/s]
Downloading: 100%|██████████| 489M/489M [00:42<00:00, 11.5MB/s] 
Downloading: 100%|██████████| 295M/295M [00:25<00:00, 11.7MB/s] 
Downloading: 100%|██████████| 309M/309M [00:26<00:00, 11.8MB/s] 
Downloading: 100%|██████████| 378M/378M [00:32<00:00, 11.6MB/s] 
Downloading: 100%|██████████| 262M/262M [00:24<00:00, 10.7MB/s] 
Downloading: 100%|██████████| 333M/333M [00:38<00:00, 8.71MB/s] 
Downloading: 100%|██████████| 383M/383M [00:35<00:00, 10.9MB/s] 
Downloading: 100%|██████████| 340M/340M [00:29<00:00, 11.7MB/s] 
Downloading: 100%|██████████| 350M/350M [00:28<00:00, 12.3MB/s] 
Downloading: 100%|██████████| 460M/460M [00:41<00:00, 11.2MB/s] 


/media/DATA-4To/Sentinel/Australie/
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20191002T232005_20191002T232305_20191004T080809_0179_050_044_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20190124T232730_20190124T233030_20190126T085154_0179_040_315_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20190123T235341_20190123T235641_20190125T072635_0179_040_301_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20191001T001227_20191001T001527_20191002T095416_0180_050_016_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20190120T233114_20190120T233414_20190122T080214_0179_040_258_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20190127T001607_20190127T001907_20190128T074734_0179_040_344_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/Australie/S3A_OL_2_WFR____20190119T235725_20190120T000025_20190121T080452_0179_040_244_3600_MAR_O_NT_002.zip
/media/DATA-4To/Sentinel/A

IndexError: index 457 is out of bounds for axis 0 with size 457